In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score, accuracy_score


from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split
import pytorch_model_summary



c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make DataFrame

## columns: review_img_path, product_img_path, label
---

In [2]:
all_product_img_list = glob.glob('./masked_data/product_img/*')
all_product_img_list = sorted(all_product_img_list)
print(all_product_img_list)

['./masked_data/product_img\\0.jpg', './masked_data/product_img\\1.jpg', './masked_data/product_img\\10.jpg', './masked_data/product_img\\100.jpg', './masked_data/product_img\\12.jpg', './masked_data/product_img\\13.jpg', './masked_data/product_img\\14.jpg', './masked_data/product_img\\15.jpg', './masked_data/product_img\\20.jpg', './masked_data/product_img\\25.jpg', './masked_data/product_img\\28.jpg', './masked_data/product_img\\30.jpg', './masked_data/product_img\\33.jpg', './masked_data/product_img\\34.jpg', './masked_data/product_img\\35.jpg', './masked_data/product_img\\36.jpg', './masked_data/product_img\\37.jpg', './masked_data/product_img\\39.jpg', './masked_data/product_img\\4.jpg', './masked_data/product_img\\40.jpg', './masked_data/product_img\\41.jpg', './masked_data/product_img\\42.jpg', './masked_data/product_img\\43.jpg', './masked_data/product_img\\45.jpg', './masked_data/product_img\\46.jpg', './masked_data/product_img\\48.jpg', './masked_data/product_img\\49.jpg', '.

In [3]:
review_img_path = []
product_img_path = []
label = []
for product_img in all_product_img_list:
    glob_img_list = []
    str = product_img.split("\\")[1] # for window
    # str = product_img.split("/")[3] # for linux
    
    str = str.split(".")[0]
    glob_img_list = glob.glob('./masked_data/review_img/'+str +'_review_img/O/*.jpg')
    review_img_path = review_img_path + glob_img_list
    product_img_path = product_img_path + [product_img  for i in range(len(glob_img_list))]

In [4]:
print(len(product_img_path))
print(len(review_img_path))


18749
18749


In [5]:
for i in range(len(review_img_path)):
    random_number = random.randint(0, 1)
    if random_number == 0:
        label.append(0)
    else:
        review_num = review_img_path[i].split('\\')[1] # for Window 
        # review_num = review_img_path[i].split('/')[3] # for Linux
        
        review_num = review_num.split("_")[0]
        random_number = random.randint(0, len(all_product_img_list) - 1)
        while review_num == random_number:
            random_number = random.randint(0, len(all_product_img_list) - 1)
        product_img_path[i] = all_product_img_list[random_number]
        label.append(1)

In [6]:
df = pd.DataFrame(columns=['review_img_path','product_img_path', 'label'])
df['review_img_path'] = review_img_path
df['product_img_path'] = product_img_path
df['label'] = label

df

,review_img_path,product_img_path,label
0,./masked_data/review_img/0_review_img/O\1.jpg,./masked_data/product_img\0.jpg,0
1,./masked_data/review_img/0_review_img/O\107.jpg,./masked_data/product_img\0.jpg,0
2,./masked_data/review_img/0_review_img/O\109.jpg,./masked_data/product_img\0.jpg,0
3,./masked_data/review_img/0_review_img/O\111.jpg,./masked_data/product_img\0.jpg,0
4,./masked_data/review_img/0_review_img/O\119.jpg,./masked_data/product_img\0.jpg,0
...,...,...,...
18744,./masked_data/review_img/98_review_img/O\93.jpg,./masked_data/product_img\79.jpg,1
18745,./masked_data/review_img/98_review_img/O\95.jpg,./masked_data/product_img\60.jpg,1
18746,./masked_data/review_img/98_review_img/O\96.jpg,./masked_data/product_img\98.jpg,0
18747,./masked_data/review_img/98_review_img/O\97.jpg,./masked_data/product_img\1.jpg,1


---

# Siames Net

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':1,
    'LEARNING_RATE':3e-3,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])


In [11]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,review_img_path,product_img_path,label,transform=None):
        self.review_img_path = review_img_path
        self.product_img_path = product_img_path
        self.label = label
        self.transform = transform
        
    def __getitem__(self,index):
        review_img = cv.imread(self.review_img_path[index])
        product_img = cv.imread(self.product_img_path[index])
        review_img = cv.resize(review_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        product_img = cv.resize(product_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        

        if self.transform is not None:
            review_img  = self.transform(image=review_img)['image']
            product_img  = self.transform(image=product_img)['image']
            
    
        return review_img, product_img, self.label[index]
    
    def __len__(self):
        return len(self.review_img_path)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])


In [13]:
train_dataset = SiameseNetworkDataset(train["review_img_path"].values, train["product_img_path"].values, train["label"].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = SiameseNetworkDataset(val["review_img_path"].values, val["product_img_path"].values, val["label"].values, train_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [14]:
avail_pretrained_models = timm.list_models('*resnet*', pretrained=True)
# avail_pretrained_models = timm.list_models(*resnet*, pretrained=True)

print(avail_pretrained_models)

['cspresnet50', 'eca_resnet33ts', 'ecaresnet26t', 'ecaresnet50d', 'ecaresnet50d_pruned', 'ecaresnet50t', 'ecaresnet101d', 'ecaresnet101d_pruned', 'ecaresnet269d', 'ecaresnetlight', 'ens_adv_inception_resnet_v2', 'gcresnet33ts', 'gcresnet50t', 'gluon_resnet18_v1b', 'gluon_resnet34_v1b', 'gluon_resnet50_v1b', 'gluon_resnet50_v1c', 'gluon_resnet50_v1d', 'gluon_resnet50_v1s', 'gluon_resnet101_v1b', 'gluon_resnet101_v1c', 'gluon_resnet101_v1d', 'gluon_resnet101_v1s', 'gluon_resnet152_v1b', 'gluon_resnet152_v1c', 'gluon_resnet152_v1d', 'gluon_resnet152_v1s', 'inception_resnet_v2', 'lambda_resnet26rpt_256', 'lambda_resnet26t', 'lambda_resnet50ts', 'legacy_seresnet18', 'legacy_seresnet34', 'legacy_seresnet50', 'legacy_seresnet101', 'legacy_seresnet152', 'nf_resnet50', 'resnet10t', 'resnet14t', 'resnet18', 'resnet18d', 'resnet26', 'resnet26d', 'resnet26t', 'resnet32ts', 'resnet33ts', 'resnet34', 'resnet34d', 'resnet50', 'resnet50_gn', 'resnet50d', 'resnet51q', 'resnet61q', 'resnet101', 'resnet1

In [15]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        # self.backbone = timm.create_model('maxvit_tiny_tf_512', pretrained=True)
        self.backbone = timm.create_model('resnetv2_152x2_bitm_in21k', pretrained=True)
        # self.backbone = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
        self.classifier = nn.Linear(1000, 10)
        
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

In [16]:
def validation(model, criterion, val_loader, device):
    model.eval()
    distance_proper = []
    distance_unfit = []
    flag = True
    
    with torch.no_grad():
        for review_img, product_img, labels in tqdm(iter(val_loader)):
            review_img = review_img.float().to(device)
            product_img = product_img.float().to(device)
            print(labels)
            if labels == 0:
                flag = True
            else:
                flag = False
                
            labels = labels.type(torch.LongTensor).to(device)
            
            pred_review = model(review_img)
            pred_product = model(product_img)
            
            if flag:
                distance_proper.append(torch.cdist(pred_review, pred_product, p=2))
            else:
                distance_unfit.append(torch.cdist(pred_review, pred_product, p=2))
                
        
    return distance_proper, distance_unfit

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model = model.to(device)
    criterion = nn.NLLLoss(reduction="sum").to(device)
    
    return validation(model, criterion, train_loader, device)

In [19]:
model = BaseModel()
model.eval()
# print(pytorch_model_summary.summary(model, torch.zeros(8,3,256,256),max_depth=None, show_parent_layers=True, show_input=True))

BaseModel(
  (backbone): ResNetV2(
    (stem): Sequential(
      (conv): StdConv2d(3, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0.0)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (stages): Sequential(
      (0): ResNetStage(
        (blocks): Sequential(
          (0): PreActBottleneck(
            (downsample): DownsampleConv(
              (conv): StdConv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (norm): Identity()
            )
            (norm1): GroupNormAct(
              32, 128, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (conv1): StdConv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): GroupNormAct(
              32, 128, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inpl

In [20]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

proper, unfit = train(model, optimizer, train_loader, val_loader, None, device)

  0%|          | 0/1641 [00:00<?, ?it/s]


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
torch.save(model.state_dict(), "./eff0b_10.pt")